# StarDist for Segmentation

---
## Learning Objectives
By the end of this module, learners will be able to:
- Understand what StarDist is and how it works conceptually.
- Choose when to use StarDist vs. Cellpose for segmentation.
- Perform segmentation using pre-trained StarDist models in Python.
- Visualize and overlay segmentation results on microscopy images.
- Identify and resolve common segmentation issues.
- Apply StarDist to a real-world dataset in a mini project.

---
## Stardist
**StarDist** is a deep learning-based instance segmentation tool specifically designed for objects with star-convex shapes (e.g., nuclei). Instead of using pixel-wise masks, **StarDist** predicts each object as a star-convex polygon, defined by a set of radial distances from the center.

**Key ideas:**
- Designed for roundish or blob-like objects such as nuclei or cells.
- Efficient and good at separating touching cells.
- Outputs instance-wise masks, not just binary foreground-background.

**Conceptual Illustration:**
Each object is represented as a star-shaped polygon radiating out from the center, making it particularly well-suited to objects with roughly circular or elliptical morphology.

### Hands-on: StarDist Installation and Imports

In [1]:
# Install StarDist and dependencies (run once)
!pip install stardist csbdeep

# Imports
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from skimage.io import imread, imshow
import matplotlib.pyplot as plt

  Using cached stardist-0.9.1.tar.gz (888 kB)


  Installing build dependencies ... -

 \

 |

 done


...

  Using cached llvmlite-0.44.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.8 kB)


Using cached csbdeep-0.8.1-py2.py3-none-any.whl (71 kB)
Using cached numba-0.61.2-cp313-cp313-macosx_11_0_arm64.whl (2.8 MB)
Using cached llvmlite-0.44.0-cp313-cp313-macosx_11_0_arm64.whl (26.2 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


 \

 error
  error: subprocess-exited-with-error
  
  × Building wheel for stardist (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [77 lines of output]
      /private/var/folders/5t/3zkcp0dd27s3txcmjn8jl96m0000gq/T/pip-build-env-mlpdfnrf/overlay/lib/python3.13/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: BSD License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
   

ModuleNotFoundError: No module named 'stardist'

### When and Why to Use StarDist (vs. Cellpose)
| Feature             | StarDist                           | Cellpose                             |
| ------------------- | ---------------------------------- | ------------------------------------ |
| Best for            | Star-shaped objects (e.g., nuclei) | Irregular shapes (e.g., whole cells) |
| Output type         | Star-convex polygons               | Vector flow + mask                   |
| Model tuning needed | Less for nucleus images            | Often more adaptable via scale       |
| Example datasets    | Fluorescent nucleus images         | Cytoplasm, brightfield, mixed cells  |



**Use StarDist if:**
- You are segmenting nuclei or small, blob-like organelles.
- You need high-quality instance segmentation.
= Your objects are mostly round or elliptical.

### Hands-on: Load and Compare Sample Images

In [ ]:
image = imread('https://raw.githubusercontent.com/stardist/stardist/master/examples/images/DSB2018/train/images/0002.tif')
plt.figure(figsize=(6,6))
plt.imshow(image, cmap='gray')
plt.title("Sample Nucleus Image")
plt.axis('off')
plt.show()

## Run Segmentation and Overlay Predictions
StarDist uses a pre-trained model (e.g., “2D_versatile_fluo”) to:
- Normalize input
- Predict polygons
- Return label masks
- The predicted instance masks can be overlaid on the original image for easy visual inspection.

### Hands-on: Apply StarDist Segmentation

In [ ]:
# Load pre-trained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# Normalize image
img_norm = normalize(image, 1, 99.8, axis=(0,1))

# Run prediction
labels, _ = model.predict_instances(img_norm)

# Show result
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(image, cmap='gray')
plt.imshow(labels, cmap='jet', alpha=0.5)
plt.title("StarDist Segmentation Overlay")
plt.axis('off')
plt.show()

---
## Troubleshooting Segmentations

Common issues and fixes:
- **Too many/few segments:** Try normalizing the input or applying Gaussian blur.
- **Merging of touching cells:** StarDist usually handles this well; check normalization.
- **Broken segment shapes:** May be due to background noise or unnormalized input.
- **Model mismatch:** Try other pre-trained models or retrain if needed.

### Hands-on: Troubleshooting Example

In [ ]:
# Try with a poorly normalized version
img_bad = image / 255.0  # Improper normalization
labels_bad, _ = model.predict_instances(img_bad)

# Compare with good normalization
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(labels_bad, cmap='jet')
plt.title("Improper Normalization")

plt.subplot(1,2,2)
plt.imshow(labels, cmap='jet')
plt.title("Proper Normalization")
plt.show()

### Exercise
Segment this image and compare it with StarDist and Cellpose.

**URL:** https://github.com/stardist/stardist/raw/master/examples/images/DSB2018/train/images/0006.tif

**Tasks:**
- Load the image.
- Run StarDist segmentation.
- Overlay predictions.
- Note any challenges or artifacts.

---
## Mini Project
**Goal:** Segment nuclei in a real fluorescence microscopy image and export results.

**Steps:**
- Download a microscopy image from your own lab or from DSB 2018 dataset.
- Apply proper normalization.
- Use StarDist to generate instance masks.
- Overlay masks on the original image.
- Save the result as an image.
- Export label data to a NumPy array or .tiff file for downstream analysis.

---
## Module Summary

| Topic                     | Key Concepts                                   | Skills Gained                                           |
| ------------------------- | ---------------------------------------------- | ------------------------------------------------------- |
| What is StarDist?         | Star-convex shapes, polygon-based segmentation | Understanding deep learning-based instance segmentation |
| StarDist vs. Cellpose     | When to use which, pros/cons                   | Selecting appropriate tools for biological images       |
| Run and visualize results | Model loading, normalization, overlays         | Applying StarDist to real images                        |
| Troubleshooting           | Normalization, overlapping, background noise   | Diagnosing and correcting segmentation problems         |
| Mini Project              | Full pipeline from image to labels             | Practical experience in bioimage analysis workflows     |
